In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, TNK test function

This is the class method for running Xopt.

TNK function
$n=2$ variables:
$x_i \in [0, \pi], i=1,2$

Objectives:
- $f_i(x) = x_i$

Constraints:
- $g_1(x) = -x_1^2 -x_2^2 + 1 + 0.1 \cos\left(16 \arctan \frac{x_1}{x_2}\right) \le 0$
- $g_2(x) = (x_1 - 1/2)^2 + (x_2-1/2)^2 \le 0.5$

In [2]:
# Import the class
from xopt import Xopt
import matplotlib.pyplot as plt
from botorch.utils.multi_objective.pareto import is_non_dominated


The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [3]:
import yaml
# Make a proper input file. 
YAML="""
xopt: {output_path: null, verbose: true}

algorithm:
  name: mobo
  options: {ref: [1.4, 1.4],
            n_initial_samples: 5,
            batch_size: 1,
            use_gpu: False,
            n_steps: 50, 
            verbose: True}
  
simulation: 
  name: test_TNK
  evaluate: xopt.evaluators.test_TNK.evaluate_TNK  
  
vocs:
  name: TNK_test
  description: null
  simulation: test_TNK
  templates: null
  variables:
    x1: [0, 3.14159]
    x2: [0, 3.14159]
  objectives: {y1: MINIMIZE, y2: MINIMIZE}
  constraints:
    c1: [GREATER_THAN, 0]
    c2: ['LESS_THAN', 0.5]
  linked_variables: {x9: x1}
  constants: {a: dummy_constant}

"""
config = yaml.safe_load(YAML)

In [4]:
# Optional: Connect the function directly
#from xopt.evaluators.test_TNK import evaluate_TNK 
#config['simulation']['evaluate'] = evaluate_TNK 

X = Xopt(config)
X

Loading config as dict.



            Xopt 
________________________________           
Version: 0.4.3
Configured: True
Config as YAML:
xopt: {output_path: null, verbose: true}
algorithm:
  name: mobo
  function: xopt.mobo.mobo
  options:
    ref: [1.4, 1.4]
    n_initial_samples: 5
    batch_size: 1
    use_gpu: false
    n_steps: 50
    verbose: true
simulation:
  name: test_TNK
  evaluate: xopt.evaluators.test_TNK.evaluate_TNK
  options: {extra_option: abc}
vocs:
  name: TNK_test
  description: null
  simulation: test_TNK
  templates: null
  variables:
    x1: [0, 3.14159]
    x2: [0, 3.14159]
  objectives: {y1: MINIMIZE, y2: MINIMIZE}
  constraints:
    c1: [GREATER_THAN, 0]
    c2: [LESS_THAN, 0.5]
  linked_variables: {x9: x1}
  constants: {a: dummy_constant}

# Run MOBO

MOBO is designed to run in serial or parallel

In [5]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

#executor = PoolExecutor()
# This will also work. 
executor=None

In [ ]:
# Change max generations
X.run(executor=executor)

Starting at time 2021-08-09T12:33:24-05:00
No executor given. Running in serial mode.
n_feas: 0
Candidate(s): tensor([[0.0667, 0.3993]], dtype=torch.float64)
n_feas: 0
Candidate(s): tensor([[0.6951, 0.9850]], dtype=torch.float64)
n_feas: 1
Candidate(s): tensor([[0., 0.]], dtype=torch.float64)
n_feas: 1
Candidate(s): tensor([[0.1795, 1.0708]], dtype=torch.float64)
n_feas: 2
Candidate(s): tensor([[0.9120, 0.3802]], dtype=torch.float64)
n_feas: 2
Candidate(s): tensor([[1.0327, 0.4501]], dtype=torch.float64)
n_feas: 3
Candidate(s): tensor([[0.0721, 1.0147]], dtype=torch.float64)
n_feas: 3
Candidate(s): tensor([[0.0688, 1.0392]], dtype=torch.float64)
n_feas: 4
Candidate(s): tensor([[0.4185, 0.8446]], dtype=torch.float64)
n_feas: 4
Candidate(s): tensor([[0.9552, 0.0366]], dtype=torch.float64)
n_feas: 4
Candidate(s): tensor([[1.0675, 0.1500]], dtype=torch.float64)
n_feas: 5
Candidate(s): tensor([[1.0208, 0.0667]], dtype=torch.float64)
n_feas: 5
Candidate(s): tensor([[1.0208, 0.0761]], dtype=t

# Plot

In [ ]:
fig, ax = plt.subplots()

# get results and get valid observations
results = X.results
train_y = results['objectives']
valid_y = train_y[results['feasibility'].flatten()]

# plot results
ax.plot(train_y[:, 0], train_y[:, 1], '.')
ax.set_ylabel('$f_2$')
ax.set_xlabel('$f_1$')

# highlight Pareto front, ONLY using valid observations (note botorch assumes maximization when determing dominant points)
non_dom = is_non_dominated(-valid_y)
ax.plot(valid_y[:,0][non_dom],valid_y[:,1][non_dom],'C1o')

plt.show()